###  Coding Challenge - Getting Started

<img align="right" style="max-width: 200px; height: auto" src="https://github.com/sandrobl/ml-eurosat/blob/main/hsg_logo.png?raw=1">
<img align="center" style="max-width: 300px; height: auto" src="https://github.com/sandrobl/ml-eurosat/blob/main/sentinel2.jpg?raw=1">

8,860,1.00 MCS Machine Learning, Spring Term 2025, University of St.Gallen (HSG)

The lab environment of the **8,860,1.00 Machine Learning** course is powered by Jupyter Notebooks (https://jupyter.org), which allows one to perform a great deal of data analysis and statistical validation. In this first lab, we want to touch on the basic concepts and techniques of such notebooks. Furthermore, its capabilities will be demonstrated based on a few simple and introductory examples.

### Objectives:

With the help of this notebook you should be able to:
    
> 1. Understand the basic funcitonality of the rasterio framework
> 2. Apply rasterio to load GTiff data
> 3. Visualize multi-band satellite imagery
> 4. Perform basic band arithmetic to compute the normalized difference vegetation index (NDVI)
> 5. Load and display samples from the challenge testset

## 2. Setup of the Jupyter Notebook Environment

Similar to the previous labs, we need to import a couple of Python libraries that allow for data analysis and data visualization. We will use `os` and `glob` to collect the filepaths for the data samples, `numpy` for array operations, `matplotlib` to display images, and `rasterio` to handle raster data. You can find the documentation of the `rasterio` library with an overview of its functionality [here](https://rasterio.readthedocs.io).

In [2]:
%pip install rasterio
%pip install matplotlib

import os
import glob
import numpy as np
import rasterio as rio
from rasterio.plot import reshape_as_image
import matplotlib.pyplot as plt
import shutil

%matplotlib inline

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [3]:

zip_file_path = "EuroSATallBands.zip"

if not os.path.exists(zip_file_path):
    print(f"File '{zip_file_path}' not found. Downloading...")
    #!powershell Invoke-WebRequest -Uri "https://madm.dfki.de/files/sentinel/EuroSATallBands.zip" -OutFile "EuroSATallBands.zip"

    # Alternative download command for Linux & MacOS
    !wget --no-check-certificate https://madm.dfki.de/files/sentinel/EuroSATallBands.zip

else:
    print(f"File '{zip_file_path}' already exists. Skipping download.")



File 'EuroSATallBands.zip' already exists. Skipping download.


## EuroSat Data Loading

First, let's collect all the files that we downloaded

In [ ]:
zip_file_path = "EuroSATallBands.zip"
extraction_path = "./data"

# Check if the extraction path exists
if not os.path.exists(extraction_path):
    shutil.unpack_archive(zip_file_path, extraction_path)
    print(f"Unpacked archive to {extraction_path}")
else:
    print(f"Folder already exists at {extraction_path}")

# change this to your eurosat path
eurosat_dir = "./data/ds/images/remote_sensing/otherDatasets/sentinel_2/tif"

Folder already exists at ./data


In [10]:
samples = glob.glob(os.path.join(eurosat_dir, "*", "*.tif"))
len(samples)

27000

We have 27,000 files across 10 classes.



# Testset Data Loading

The testset has a slightly different structure than Eurosat. There are no labels and the data is stored in `numpy` `.npy` instead of GTiff.

First, we have to download the data from [Kaggle](https://www.kaggle.com/competitions/8-860-1-00-coding-challenge-2025/data).

In [11]:
zip_test_file_path = "./testset.zip"
extraction_path = "./data"

testset_dir = os.path.join(extraction_path, "testset")

if not os.path.exists(zip_test_file_path):
    print(f"File '{zip_test_file_path}' not found. Will download the testdata here")
    !wget https://github.com/sandrobl/ml-eurosat/raw/main/testset.zip

    #Windows Powershell version
    #!powershell Invoke-WebRequest -Uri "https://github.com/sandrobl/ml-eurosat/raw/main/testset.zip" -OutFile "testset.zip"

else:
    print(f"File '{zip_test_file_path}' already exists. Skipping download.")


# Check if the testset directory exists
if not os.path.exists(testset_dir):
    shutil.unpack_archive(zip_test_file_path, extraction_path)
    print(f"Unpacked archive to {extraction_path}")
else:
    print(f"Testset folder already exists at {testset_dir}")


test_samples = glob.glob(os.path.join(testset_dir, "*.npy"))
len(test_samples)

File './testset.zip' already exists. Skipping download.
Unpacked archive to ./data


4232

## Next steps

Create a (deep learning) model to predict the most likely Eurosat class for each image of the testset. Think about creating the dataset class and data-loader for training, possible model architectures, and perhaps even how to best address the shift between train and test data.

In [ ]:
%pip install torch torchvision

import os
import rasterio
import torch
import torch.nn as nn
import torchvision.models as models  # Import the missing module
import torch.optim as optim

from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image

classes = [
    "AnnualCrop",
    "Forest",
    "HerbaceousVegetation",
    "Highway",
    "Industrial",
    "Pasture",
    "PermanentCrop",
    "Residential",
    "River",
    "SeaLake",
]

def normalize_for_display(band_data):
    band_data = np.array(band_data)
    lower_perc = np.percentile(band_data, 2, axis=(0,1))
    upper_perc = np.percentile(band_data, 98, axis=(0,1))
    return (band_data - lower_perc) / (upper_perc - lower_perc)


# Add the EurosatDataset class
class EurosatDataset(torch.utils.data.Dataset):
    def __init__(self, samples, transform=None):
        self.samples = samples
        self.transform = transform
        self.classes = classes

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample_path = self.samples[idx]
        #label = sample_path.split('/')[-1].split('_')[0]
        label = os.path.basename(os.path.dirname(sample_path))

        label_idx = self.classes.index(label)

        with rio.open(sample_path, "r") as d:
            img = d.read([1,2,3,4,5,6,7,8,9,10,11,12,13])
            img = reshape_as_image(img)
            img = normalize_for_display(img)
            img = img[:, :, [3,2,1]]  # Extract RGB bands

        if self.transform:
            img = self.transform(img)

        return img, label_idx

# Define transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create dataset instances
train_dataset = EurosatDataset(samples, transform=transform)
test_dataset = EurosatDataset(test_samples, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

class EurosatModel(nn.Module):  # Fix inheritance (nn.Module)
    def __init__(self, num_classes=10):
        super(EurosatModel, self).__init__()  # Fix super() syntax
        self.model = models.resnet50(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):  # Fix indentation
        return self.model(x)

model = EurosatModel(num_classes=10)


# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 15
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 10 == 9:  # Print every 10 batches
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 10:.4f}')
            running_loss = 0.0

print('Finished Training')

torch.save(model.state_dict(), "model_checkpoint.pth")

# ----------------------------------------------
# 1. Define the EurostatDataset Class (REQUIRED)
# ----------------------------------------------
class EurostatDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted([cls for cls in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, cls))])
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}
        self.images = []

        # Iterate through all subfolders (classes) in the root directory
        for cls in self.classes:
            cls_dir = os.path.join(root_dir, cls)  # Path to the class directory
            # Iterate through all files in the class directory
            for img_name in os.listdir(cls_dir):
                if img_name.endswith('.tif'):  # Ensure images are .tif files
                    # Append the image path and label to the images list
                    self.images.append((os.path.join(cls_dir, img_name), self.class_to_idx[cls]))

    def __len__(self):
        # Return the total number of images in the dataset
        return len(self.images)

    def __getitem__(self, idx):
        # Get the image path and label for the given index
        img_path, label = self.images[idx]

        # Open the image using rasterio and read it as a numpy array
        with rasterio.open(img_path) as src:
            image = src.read([4, 3, 2])  # Read RGB bands (R=4, G=3, B=2)
            image = image.transpose(1, 2, 0)  # Change to HxWxC

        # Convert numpy array to PIL Image
        image = Image.fromarray(image.astype('uint8'), 'RGB')

        # Apply any transformations to the image
        if self.transform:
            image = self.transform(image)
        # Return the transformed image and its label
        return image, label
# ----------------------------------------------
# 2. Define Transforms and Load Test Data
# ----------------------------------------------
# Use the same transforms as during training
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize images to 64x64
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])

# Load Test Dataset
# Note: The root directory is corrected to point to the 'testset' folder
test_dataset = EurostatDataset(
    root_dir='./data/testset',
    transform=transform
)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# ----------------------------------------------
# 3. Evaluation Loop
# ----------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval()  # Set the model to evaluation mode

correct = 0
total = 0

with torch.no_grad():  # Disable gradient calculation during evaluation
    for inputs, labels in test_loader:
        # Move inputs and labels to the appropriate device
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Get model predictions
        outputs = model(inputs)
        # Get the predicted class indices
        _, predicted = torch.max(outputs.data, 1)

        # Update total and correct counts
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate and print the test accuracy
print(f'Test Accuracy: {100 * correct / total:.2f}%')



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
c:\Users\Sandro\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Sandro\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


True
Epoch [1/5], Step [10/844], Loss: 1.5434
Epoch [1/5], Step [20/844], Loss: 1.1680
Epoch [1/5], Step [30/844], Loss: 1.3435
Epoch [1/5], Step [40/844], Loss: 1.0486
Epoch [1/5], Step [50/844], Loss: 1.0225
Epoch [1/5], Step [60/844], Loss: 0.7460
Epoch [1/5], Step [70/844], Loss: 0.7868
Epoch [1/5], Step [80/844], Loss: 0.8435
Epoch [1/5], Step [90/844], Loss: 0.7198
Epoch [1/5], Step [100/844], Loss: 0.7753
Epoch [1/5], Step [110/844], Loss: 0.8235
Epoch [1/5], Step [120/844], Loss: 0.6877
Epoch [1/5], Step [130/844], Loss: 0.5918
Epoch [1/5], Step [140/844], Loss: 0.6357
Epoch [1/5], Step [150/844], Loss: 0.5178


C:\Users\Sandro\AppData\Local\Temp\ipykernel_15844\1476928379.py:31: RuntimeWarning: invalid value encountered in divide
  return (band_data - lower_perc) / (upper_perc - lower_perc)


Epoch [1/5], Step [160/844], Loss: 0.6806


KeyboardInterrupt: 